# 1) Main

## 1.1) Connect Google Drive with Google Colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.2) Install Nilearn

In [ ]:
!pip install nilearn

## 1.3) Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
import errno

import nilearn as nl
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1.4) Define the path where this notebook is located <-- **EDIT !**




For example, if this notebook is inside a folder with the name `benchmark`, in your google drive, the path is defined as:

> `path_main = '/content/drive/MyDrive/benchmark'`

As shown in the example, you should NOT add a slash (/) at the end of the indicated path.

In [ ]:
###################### EDIT ! #########################
path_main = '/content/drive/MyDrive/myFolder'
#######################################################

# Check if the path is correct
notebook_name = 'machine_learning_benchmark.ipynb'
if os.path.isfile(path_main+'/'+ notebook_name):
  print('Correct path') 
else:
  print('Error: incorrect path')
  print('- Check if this notebook is within the indicated path')
  print('- Check the name of this notebook is: '+notebook_name)    

# 2) Functions

In this notebook, the Preprocessed Connectomes Project (PCP) nomenclature is used to define the four strategies, which differs from that of our paper. Therefore, the following lines show the equivalent between the PCP nomenclature (left) and that of our paper (right).


> `filt_global = BPF+GSR`

> `filt_noglobal = BPF`

> `nofilt_global = GSR`

> `nofilt_noglobal = none`

## 2.1) Define strategy using a Boolean vector

This function returns a Boolean vector to identify the desired strategy when downloading the data.

In [ ]:
def bool_strategy(name_strategy):
  if name_strategy == 'filt_global':
    bool_vector = np.array([True, True], dtype=bool)
  if name_strategy == 'filt_noglobal':
    bool_vector = np.array([True, False], dtype=bool)
  if name_strategy == 'nofilt_global':
    bool_vector = np.array([False, True], dtype=bool)
  if name_strategy == 'nofilt_noglobal':
    bool_vector = np.array([False, False], dtype=bool) 
    
  return bool_vector

## 2.2) Extract features

This function allows extracting the features of the cases and storing them in a file with extension `.npz`. Specifically, for each case, a one-dimensional vector of features with length `L = 2016` is obtained.

In [ ]:
def features_all(path_output, strategy, path_cases):
  name_save = strategy+'_features_all.npz'
  
  # Load atlas
  multiscale = datasets.fetch_atlas_basc_multiscale_2015()
  atlas_filename = multiscale.scale064
  
  # Initialize masker object
  masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True, verbose=0)
  
  # Initialize correlation measure
  correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True, discard_diagonal=True)

  print('\n----> Strategy: '+strategy)

  try: # Check if the feature file exists
    # Load features
    feat_file = os.path.join(path_output, name_save)
    features = np.load(feat_file)['a']
    print("Feature file found")

  except: # If not, extract features
    features = []
    print("No feature file found. Extracting features ...")

    for i,case in enumerate(path_cases):
        # Extract the time series from the ROI's
        time_series = masker.fit_transform(case)      
        
        # Create correlation matrix
        correlation_matrix = correlation_measure.fit_transform([time_series])[0]
        
        features.append(correlation_matrix)
        print('Features %s of %s extracted'%(i+1,len(path_cases)))

    # Save features
    np.savez_compressed(os.path.join(path_output, name_save), a = features)

## 2.3) Get target

This function returns two vectors (`train_target` and `validation_target`) containing the labels (`1` for autism or `2` for normal patient) of the cases used in the `train` and `validation` sets defined in each fold.

In [ ]:
def get_target(path_main, fold):
  with open(path_main+'/folds.pickle', 'rb') as op1:
    folds = pickle.load(op1)
  path_dxgroup = pd.read_csv(path_main+'/path_dxgroup.csv')
  train_target = np.array([(path_dxgroup['dx_group'])[i] for i,case in enumerate(path_dxgroup['file_id']) if case not in folds['fold'+str(fold)]])
  validation_target = np.array([(path_dxgroup['dx_group'])[i] for i,case in enumerate(path_dxgroup['file_id']) if case in folds['fold'+str(fold)]])
  
  return train_target, validation_target

## 2.4) Grid search method

This function uses the grid search method to find the best model for each machine learning algorithm. In this way, it returns two variables corresponding to the best model and the best parameters found, respectively.

In [ ]:
def best_estimator(model, param_grid, features, target, cv):
  grid = GridSearchCV(model, param_grid, cv=cv)
  grid.fit(features, target)
  best_model = grid.best_estimator_
  best_param = grid.best_params_
  
  return best_model, best_param

# 3) Download preprocessed rs-fMRI data


At this point the rs-fMRI data are downloaded according to the preprocessing strategy. Specifically, data from the NYU site through the CPAC pipeline are considered.

In [ ]:
path_donwload = path_main
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']

for strategy in strategies:
  bool_vector = bool_strategy(strategy)
  abide_asd = datasets.fetch_abide_pcp(data_dir = path_donwload, pipeline = "cpac", band_pass_filtering = bool_vector[0],                               
                                   global_signal_regression = bool_vector[1], quality_checked = False, SITE_ID=['NYU'], DX_GROUP=['1'])
  
  abide_tc = datasets.fetch_abide_pcp(data_dir = path_donwload, pipeline = "cpac", band_pass_filtering = bool_vector[0],                               
                                   global_signal_regression = bool_vector[1], quality_checked = False, SITE_ID=['NYU'], DX_GROUP=['2'])
   
  abide_asd = abide_tc = []

# 4) Paths, id and dx_group of the cases

This section allows to create a file with `.csv` extension containing the `paths` of the cases, depending on the strategy, the `id` (e.g., NYU_0050952) and the `dx_group` (1 is a patient with autism and 2 is a normal patient).

In [ ]:
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']

extra_pheno = ['file_id', 'dx_group']
pheno = pd.read_csv(path_main+'/ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv')
fileid_dxgroup = [[pheno.FILE_ID[i], pheno.DX_GROUP[i]] for i in range(len(pheno)) if (pheno.FILE_ID[i])[0:4] == 'NYU_']
path_dxgroup = pd.DataFrame(columns=strategies+extra_pheno)
for strategy in strategies:
  path_dxgroup[strategy] = [path_main+'/ABIDE_pcp/cpac/'+strategy+'/'+str(i[0])+'_func_preproc.nii.gz' for i in fileid_dxgroup]
path_dxgroup['file_id'] = [str(i[0]) for i in fileid_dxgroup]
path_dxgroup['dx_group'] = [str(i[1]) for i in fileid_dxgroup]
path_dxgroup.to_csv(path_main+'/path_dxgroup.csv') 

# 5) Feature extraction

## 5.1) Features of all cases according to the strategy

In this section we obtain the characteristics of all cases for each preprocessing strategy.

In [ ]:
path_output = path_main+'/features'
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']

try:
  os.mkdir(path_output)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

path_cases = pd.read_csv(path_main+'/path_dxgroup.csv')
for strategy in strategies:
  features_all(path_output, strategy, path_cases[strategy])  

## 5.2) Features according to strategy and fold

In this section the files are created, by strategy, containing the characteristics of the cases belonging to the train or validation sets defined in each fold.

In [ ]:
path_output = path_main+'/features'
num_folds = 5
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']

with open(path_main+'/folds.pickle', 'rb') as op1:
    folds = pickle.load(op1)

path_dxgroup = pd.read_csv(path_main+'/path_dxgroup.csv')

for strategy in strategies:
  print('----> Strategy: '+strategy)
  features_each_fold = {}
  features = np.load(path_output+'/'+strategy+'_features_all.npz')['a']
  for fold in range(1, num_folds+1):
    features_train = np.array([features[i,:] for i,case in enumerate(path_dxgroup['file_id']) if case not in folds['fold'+str(fold)]])
    features_validation = np.array([features[i,:] for i,case in enumerate(path_dxgroup['file_id']) if case in folds['fold'+str(fold)]])
    features_each_fold['fold'+str(fold)+'_train'] = features_train
    features_each_fold['fold'+str(fold)+'_validation'] = features_validation
  np.savez_compressed(os.path.join(path_output, strategy+'_features_folds'), **features_each_fold)
  features = features_each_fold = []    

# 6) Machine learning algorithms

## 6.1) Support Vector Machines (SVM)

In [ ]:
path_output = path_main+'/features'
path_models = path_main+'/models'
num_folds = 5
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']

try:
  os.mkdir(path_models)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

measures_svm_strtg = []

for strategy in strategies:
  print('\n----> Strategy: '+strategy)
  measures_svm_folds = []

  for fold in range(1, num_folds+1):
    print('Fold: '+str(fold))
    file_features = np.load(os.path.join(path_output, strategy+'_features_folds.npz'))
    train_features = file_features['fold'+str(fold)+'_train']
    validation_features = file_features['fold'+str(fold)+'_validation']
    train_target, validation_target = get_target(path_main, fold)
    model_svm = SVC()

    try: # Check if the file containing the best model exists
      # Load best model
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_svm.pickle', 'rb') as om:
        best_model_svm = pickle.load(om)
      
    except: # If not, search for the best model
      param_grid_svm = {'C': [0.01, 0.1, 1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4,  5, 10],
                        'gamma': [0.00001, 0.00005,  0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 1.5, 2, 3.5, 5, 10]}

      _, best_params_svm = best_estimator(model_svm, param_grid_svm, train_features, train_target, 10)

      best_model_svm = SVC(C=best_params_svm['C'], break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
                          decision_function_shape='ovr', degree=3, gamma=best_params_svm['gamma'], kernel='rbf',
                          max_iter=-1, probability=False, random_state=None, shrinking=True,
                          tol=0.001, verbose=False)

      # Save best_model_svm
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_svm.pickle', 'wb') as ff1:
          pickle.dump(best_model_svm, ff1)

    clf_svm = best_model_svm.fit(train_features, train_target)
    pred_target = clf_svm.predict(validation_features)

    pred_target[pred_target==2] = 0
    validation_target[validation_target==2] = 0
    
    TN, FP, FN, TP = (confusion_matrix(validation_target, pred_target).ravel()).astype(float)

    accuracy = (TP+TN)/(TP+TN+FP+FN)
    specificity = TN/(FP+TN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    fscore = 2*TP/((2*TP)+FP+FN)

    measures_svm = [accuracy, precision, fscore, recall, specificity]

    measures_svm_folds.append(measures_svm)

    # Delete content of some variables
    model_SVC = train_features = validation_features = train_target = validation_target = []
    om = ff1 = best_model_svm = best_params_svm = clf_svm = pred_target = measures_svm = []

  measures_svm_strtg.append(np.array(measures_svm_folds).mean(axis=0)) 

### Save results - SVM

In [ ]:
path_results = path_main+'/results'

try:
  os.mkdir(path_results)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

# Save measures
with open(path_results+'/measures_svm.pickle', 'wb') as ff1:
    pickle.dump(measures_svm_strtg, ff1)

ff1 = []

### Table of results - SVM

In [ ]:
measures_names = ['Accuracy', 'Precision', 'F-score', 'Recall', 'Specificity']
table_svm = pd.DataFrame(columns=measures_names, index=strategies)
measures_svm_strtg = (np.array(measures_svm_strtg)*1000).astype(int)/1000
pd.options.display.float_format = '{:,.3f}'.format

for i, strategy in enumerate(strategies):
  table_svm.loc[strategy] = measures_svm_strtg[i]

table_svm

,Accuracy,Precision,F-score,Recall,Specificity
filt_global,0.680,0.685,0.556,0.480,0.830
filt_noglobal,0.685,0.701,0.536,0.440,0.870
nofilt_global,0.697,0.698,0.605,0.546,0.809
nofilt_noglobal,0.645,0.668,0.483,0.386,0.840


## 6.2) K-Neirest Neighbor (KNN)

In [ ]:
path_output = path_main+'/features'
path_models = path_main+'/models'
num_folds = 5
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']


try:
  os.mkdir(path_models)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

measures_knn_strtg = [] 

for strategy in strategies:
  print('\n----> Strategy: '+strategy)
  measures_knn_folds = []

  for fold in range(1, num_folds+1):
    print('Fold: '+str(fold))
    file_features = np.load(os.path.join(path_output, strategy+'_features_folds.npz'))
    train_features = file_features['fold'+str(fold)+'_train']
    validation_features = file_features['fold'+str(fold)+'_validation']
    train_target, validation_target = get_target(path_main, fold)
    model_knn = KNN()

    try: # Check if the file containing the best model exists
      # Load best model
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_knn.pickle', 'rb') as om:
        best_model_knn = pickle.load(om)    

    except: # If not, search for the best model 
      param_grid_knn = {'n_neighbors': [1, 2 , 3, 4, 6, 10, 15, 20, 25, 30, 50, 100],
                        'algorithm':['auto', 'kd_tree']}

      _, best_params_knn = best_estimator(model_knn, param_grid_knn, train_features, train_target, 10)

      best_model_knn = KNN(algorithm=best_params_knn['algorithm'], leaf_size=30, metric='minkowski',
                          metric_params=None, n_jobs=None, n_neighbors=best_params_knn['n_neighbors'], p=2,
                          weights='uniform')

      # Save best_model_knn
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_knn.pickle', 'wb') as ff1:
          pickle.dump(best_model_knn, ff1)

    clf_knn = best_model_knn.fit(train_features, train_target)
    pred_target = clf_knn.predict(validation_features)

    pred_target[pred_target==2] = 0
    validation_target[validation_target==2] = 0

    TN, FP, FN, TP = (confusion_matrix(validation_target, pred_target).ravel()).astype(float)

    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    fscore = 2*TP/((2*TP)+FP+FN)
    recall = TP/(TP+FN)
    specificity = TN/(FP+TN)

    measures_knn = [accuracy, precision, fscore, recall, specificity]

    measures_knn_folds.append(measures_knn)

    # Delete content of some variables
    model_knn = train_features = validation_features = train_target = validation_target = []
    om = ff1 = best_model_knn = best_params_knn = clf_knn = pred_target = measures_knn = []

  measures_knn_strtg.append(np.array(measures_knn_folds).mean(axis=0))

### Save results - KNN

In [ ]:
path_results = path_main+'/results'

try:
  os.mkdir(path_results)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

# Save measures
with open(path_results+'/measures_knn.pickle', 'wb') as ff1:
    pickle.dump(measures_knn_strtg, ff1)

ff1 = []

### Table of results - KNN

In [ ]:
measures_names = ['Accuracy', 'Precision', 'F-score', 'Recall', 'Specificity']
table_knn = pd.DataFrame(columns=measures_names, index=strategies)
measures_knn_strtg = (np.array(measures_knn_strtg)*1000).astype(int)/1000
pd.options.display.float_format = '{:,.3f}'.format

for i, strategy in enumerate(strategies):
  table_knn.loc[strategy] = measures_knn_strtg[i]

table_knn

,Accuracy,Precision,F-score,Recall,Specificity
filt_global,0.639,0.618,0.509,0.440,0.790
filt_noglobal,0.600,0.537,0.437,0.386,0.760
nofilt_global,0.634,0.600,0.501,0.439,0.780
nofilt_noglobal,0.600,0.556,0.487,0.453,0.710


## 6.3) Decision Tree (DT)

In [ ]:
path_output = path_main+'/features'
path_models = path_main+'/models'
num_folds = 5
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']


try:
  os.mkdir(path_models)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

measures_dt_strtg = []

for strategy in strategies:
  print('\n----> Strategy: '+strategy)
  measures_dt_folds = []

  for fold in range(1, num_folds+1):
    print('Fold: '+str(fold))
    file_features = np.load(os.path.join(path_output, strategy+'_features_folds.npz'))
    train_features = file_features['fold'+str(fold)+'_train']
    validation_features = file_features['fold'+str(fold)+'_validation']
    train_target, validation_target = get_target(path_main, fold)   
    model_dt = DecisionTreeClassifier(random_state=1)

    try: # Check if the file containing the best model exists
      # Load best model
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_dt.pickle', 'rb') as om:
        best_model_dt = pickle.load(om)   

    except: # If not, search for the best model
      param_grid_dt = {'max_depth': [1, 2 , 3, 4, 5, 6, 10]}

      _ , best_params_dt = best_estimator(model_dt, param_grid_dt, train_features, train_target, 10)

      best_model_dt = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                              max_depth=best_params_dt['max_depth'], max_features=None, max_leaf_nodes=None,
                              min_impurity_decrease=0.0, min_impurity_split=None,
                              min_samples_leaf=1, min_samples_split=2,
                              min_weight_fraction_leaf=0.0, presort='deprecated',
                              random_state=1, splitter='best')

      # Save model_best tree
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_dt.pickle', 'wb') as ff1:
          pickle.dump(best_model_dt, ff1)

    clf_dt = best_model_dt.fit(train_features, train_target)
    pred_target = clf_dt.predict(validation_features)

    pred_target[pred_target==2] = 0
    validation_target[validation_target==2] = 0

    TN, FP, FN, TP = (confusion_matrix(validation_target, pred_target).ravel()).astype(float)

    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    fscore = 2*TP/(2*TP+FP+FN)
    recall = TP/(TP+FN)
    specificity = TN/(FP+TN)

    measures_dt = [accuracy, precision, fscore, recall, specificity]

    measures_dt_folds.append(measures_dt)

    # Delete content of some variables
    model_dt = train_features = validation_features = train_target = validation_target = []
    om = ff1 = best_model_dt = best_params_dt = clf_dt = pred_target = measures_dt = []

  measures_dt_strtg.append(np.array(measures_dt_folds).mean(axis=0))

### Save results - DT

In [ ]:
path_results = path_main+'/results'

try:
  os.mkdir(path_results)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

# Save measures
with open(path_results+'/measures_dt.pickle', 'wb') as ff1:
    pickle.dump(measures_dt_strtg, ff1)

ff1 = []

### Table of results - DT

In [ ]:
measures_names = ['Accuracy', 'Precision', 'F-score', 'Recall', 'Specificity']
table_dt = pd.DataFrame(columns=measures_names, index=strategies)
measures_dt_strtg = (np.array(measures_dt_strtg)*1000).astype(int)/1000
pd.options.display.float_format = '{:,.3f}'.format

for i, strategy in enumerate(strategies):
  table_dt.loc[strategy] = measures_dt_strtg[i]

table_dt

,Accuracy,Precision,F-score,Recall,Specificity
filt_global,0.565,0.527,0.513,0.533,0.590
filt_noglobal,0.559,0.485,0.486,0.493,0.610
nofilt_global,0.628,0.603,0.500,0.440,0.770
nofilt_noglobal,0.571,0.495,0.476,0.466,0.650


## 6.4) Random Forests (RF)

In [ ]:
path_output = path_main+'/features'
path_models = path_main+'/models'
num_folds = 5
strategies = ['filt_global', 'filt_noglobal', 'nofilt_global', 'nofilt_noglobal']


try:
  os.mkdir(path_models)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

measures_rf_strtg = []

for strategy in strategies:
  print('\n----> Strategy: '+strategy)
  measures_rf_folds = [] 

  for fold in range(1, num_folds+1):
    print('Fold: '+str(fold))
    file_features = np.load(os.path.join(path_output, strategy+'_features_folds.npz'))
    train_features = file_features['fold'+str(fold)+'_train']
    validation_features = file_features['fold'+str(fold)+'_validation']
    train_target, validation_target = get_target(path_main, fold)
    model_rf = RandomForestClassifier(random_state=42)

    try: # Check if the file containing the best model exists
      # Load best model
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_rf.pickle', 'rb') as om:
        best_model_rf = pickle.load(om)   

    except: # If not, search for the best model    
      param_grid_rf = {'n_estimators': [100, 200, 500, 700],
                       'max_features': ['sqrt', 'log2']}    

      best_model_rf, best_params_rf = best_estimator(model_rf, param_grid_rf, train_features, train_target, 10)

      # Save best_model_rf
      with open(path_models+'/'+strategy+'_fold'+str(fold)+'_best_model_rf.pickle', 'wb') as ff1:
          pickle.dump(best_model_rf, ff1)

    clf_rf = best_model_rf.fit(train_features, train_target)
    pred_target = clf_rf.predict(validation_features)

    pred_target[pred_target==2] = 0
    validation_target[validation_target==2] = 0

    TN, FP, FN, TP = (confusion_matrix(validation_target, pred_target).ravel()).astype(float)

    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    fscore = 2*TP/((2*TP)+FP+FN)
    recall = TP/(TP+FN)
    specificity = TN/(FP+TN)

    measures_rf = [accuracy, precision, fscore, recall, specificity]

    measures_rf_folds.append(measures_rf)

    # Delete content of some variables
    model_rf = train_features = validation_features = train_target = validation_target = []
    om = ff1 = model_best_rf = best_params_rf = clf_rf = pred_target = measures_rf = []

  measures_rf_strtg.append(np.array(measures_rf_folds).mean(axis=0))

### Save results - RF

In [ ]:
path_results = path_main+'/results'

try:
  os.mkdir(path_results)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

with open(path_results+'/measures_rf.pickle', 'wb') as ff1:
    pickle.dump(measures_rf_strtg, ff1)

ff1 = []

### Table of results - RF

In [ ]:
measures_names = ['Accuracy', 'Precision', 'F-score', 'Recall', 'Specificity']
table_rf = pd.DataFrame(columns=measures_names, index=strategies)
measures_rf_strtg = (np.array(measures_rf_strtg)*1000).astype(int)/1000
pd.options.display.float_format = '{:,.3f}'.format

for i, strategy in enumerate(strategies):
  table_rf.loc[strategy] = measures_rf_strtg[i]

table_rf

,Accuracy,Precision,F-score,Recall,Specificity
filt_global,0.617,0.640,0.410,0.319,0.840
filt_noglobal,0.674,0.734,0.505,0.386,0.889
nofilt_global,0.679,0.740,0.507,0.386,0.900
nofilt_noglobal,0.651,0.725,0.468,0.360,0.869
